In [4]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
import matplotlib.gridspec as gridspec
from scipy.io import arff
from sklearn.preprocessing import LabelEncoder

In [5]:
artists_path = os.path.join('D:\\', 'artists')
original_datasets_path = os.path.abspath(artists_path)

# Load data
def load_data(artists_datasets_path: str):
    artist_data = pd.read_csv(artists_datasets_path, delimiter=",")  # Intenta también con "\t"
    return artist_data


# Load data 
artists_datasets_path = os.path.join(artists_path, 'result_df_unique_reset_year.csv')
result_df_unique_reset_year = load_data(artists_datasets_path)

# Show first records and DataFrame information
display(result_df_unique_reset_year.head())


,e.country_3,e.startdate,num_exhibitions
0,AUT,1904,2
1,AUT,1905,294
2,AUT,1906,488
3,AUT,1907,349
4,AUT,1908,605


In [6]:
import numpy as np
import plotly.express as px


# Agregar una columna transformada logarítmicamente
result_df_unique_reset_year['log_num_exhibitions'] = np.log1p(result_df_unique_reset_year['num_exhibitions'])

# Calcular el rango para la escala logarítmica
log_color_min = result_df_unique_reset_year['log_num_exhibitions'].min()
log_color_max = result_df_unique_reset_year['log_num_exhibitions'].max()

# Crear el gráfico utilizando la columna transformada
fig = px.choropleth(result_df_unique_reset_year, 
                    locations="e.country_3",
                    color="log_num_exhibitions",  # Usar la columna logarítmica
                    hover_name="e.country_3",
                    hover_data={"log_num_exhibitions": False,  # Ocultar valores logarítmicos en el hover
                                "num_exhibitions": True,  # Mostrar valores originales
                                "e.startdate": True},
                    animation_frame="e.startdate",
                    color_continuous_scale=px.colors.sequential.Plasma,
                    range_color=[log_color_min, log_color_max],  # Fijar el rango para la escala logarítmica
                    projection="natural earth",
                    title="Exhibition Distribution by Country and Year (Log Scale)"
)

# Personalizar el mapa
fig.update_geos(showcoastlines=True, coastlinecolor="Black", showland=True, landcolor="lightgray")
fig.update_layout(title_text="Exhibition Distribution by Country and Year (Log Scale)", title_x=0.5)

# Guardar y mostrar el gráfico
fig.write_html("interactive_map_log_scale.html")
fig.show()



In [7]:
import numpy as np
import plotly.express as px

# Add a logarithmic transformed column
result_df_unique_reset_year['log_num_exhibitions'] = np.log1p(result_df_unique_reset_year['num_exhibitions'])

# Calculate the range for the logarithmic scale
log_color_min = result_df_unique_reset_year['log_num_exhibitions'].min()
log_color_max = result_df_unique_reset_year['log_num_exhibitions'].max()

# Add historical events for animation frames
event_dict = {
    "1912-04": "Titanic Sinking",
    "1914-07": "Start of WWI",
    "1918-11": "End of WWI"
}

# Create a new column to display events alongside dates in the animation frame
result_df_unique_reset_year['animation_frame_with_events'] = result_df_unique_reset_year['e.startdate'].apply(
    lambda x: f"{x} ({event_dict[x]})" if x in event_dict else x
)

# Create the choropleth map using the transformed column
fig = px.choropleth(
    result_df_unique_reset_year, 
    locations="e.country_3",
    color="log_num_exhibitions",  # Use the logarithmic column
    hover_name="e.country_3",
    hover_data={"log_num_exhibitions": False,  # Hide logarithmic values in hover
                "num_exhibitions": True,  # Show original values
                "e.startdate": True},
    animation_frame="animation_frame_with_events",  # Use the new column with events
    color_continuous_scale=px.colors.sequential.Plasma,
    range_color=[log_color_min, log_color_max],  # Fix the range for the logarithmic scale
    projection="natural earth",
    title="Exhibition Distribution by Country and Year (Log Scale)"
)

# Customize the map
fig.update_geos(showcoastlines=True, coastlinecolor="Black", showland=True, landcolor="lightgray")
fig.update_layout(
    title_text="Exhibition Distribution by Country and Year (Log Scale)", 
    title_x=0.5,
    sliders=[{
        "steps": [
            {"label": f"{frame.name}", "method": "animate", "args": [[frame.name], {}]}
            for frame in fig.frames
        ],
        "currentvalue": {"prefix": "Year: "},
        "pad": {"t": 50},
        "x": 0.1, "len": 0.9,  # Position slider
        "active": 0
    }]
)

# Add titles above the slider
fig.add_annotation(
    x=0.6, y=-0.35,  # Adjust position
    xref="paper", yref="paper",
    text="1912: Titanic Sinks",
    showarrow=False,
    font=dict(size=12, color="black"),
    align="center"
)
fig.add_annotation(
    x=0.78, y=-0.35,  # Adjust position
    xref="paper", yref="paper",
    text="1914: Start of WWI",
    showarrow=False,
    font=dict(size=12, color="black"),
    align="center"
)

# Save and display the chart
fig.write_html("interactive_map_log_scale_with_events.html")
fig.show()

